In [1]:
import pandas as pd 
import numpy as np
import copy
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('Heart_disease_cleveland_new.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [4]:
class BCNN:
    def __init__(self, layers, epochs, verbose =True, learning_rate = 0.1, seed = 42):
        np.random.seed(seed)
        self.learning_rate = learning_rate
        self.layers = layers
        self.epochs = epochs
        self.verbose = verbose
        
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))

    def derivative_sigmoid(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def cross_entropy_loss(self, y_true, y_pred):
        y_pred = np.clip(y_pred, 1e-8, 1-1e-8)
        return -np.mean(y_true * np.log(y_pred) + (1-y_true) * np.log(1-y_pred))

    def forward(self, X):
        self.linear_combinations = []
        self.activations = []
        
        self.linear_combinations.append(np.dot(X, self.weights[0]) + self.biases[0])
        self.activations.append(self.sigmoid(self.linear_combinations[0]))
        
        for i in range(len(self.layers)-1):
            self.linear_combinations.append(np.dot(self.activations[i], self.weights[i+1]) + self.biases[i+1])
            self.activations.append(self.sigmoid(self.linear_combinations[i+1]))

        self.linear_combinations.append(np.dot(self.activations[-1], self.weights[-1]) + self.biases[-1])
        self.activations.append(self.sigmoid(self.linear_combinations[-1]))

        return self.activations[-1]

    def backward(self, X, y):
        m = X.shape[0]
        self.deltas = []
        dz = self.activations[-1] - y
        dw = np.dot(self.activations[-2].T,dz)/m
        db = np.sum(dz,axis = 0, keepdims = True)/m
        self.deltas.append((dz,dw,db))

        for i, (w,z,a) in enumerate(zip(self.weights[::-1][:-2],self.linear_combinations[::-1][1:-1], self.activations[::-1][2:])):
            dz = np.dot(self.deltas[i][0], w.T) * self.derivative_sigmoid(z)
            dw = np.dot(a.T, dz)/m
            db = np.sum(dz,axis = 0, keepdims = True)/m
            self.deltas.append((dz,dw,db))

        dz = np.dot(self.deltas[-1][0], self.weights[1].T) * self.derivative_sigmoid(self.linear_combinations[0])
        dw = np.dot(X.T,dz)/m
        db = np.sum(dz,axis = 0, keepdims = True)/m
        self.deltas.append((dz,dw,db))

        for i,d in enumerate(self.deltas[::-1]):
            self.weights[i] -= self.learning_rate * d[1]
            self.biases[i] -= self.learning_rate * d[2]
        
    def fit(self, X, y):

        self.weights = []
        self.biases = []
        
        self.weights.append(np.random.randn(X.shape[1],self.layers[0]))
        self.biases.append(np.zeros((1,self.layers[0])))
        
        for i in range(len(self.layers)-1):
            self.weights.append(np.random.randn(self.layers[i],self.layers[i+1]))
            self.biases.append(np.zeros((1,self.layers[i+1])))

        self.weights.append(np.random.randn(self.layers[-1],1))
        self.biases.append(np.zeros((1,1)))
        
        for epoch in range(self.epochs):
            y_pred = self.forward(X)
            loss = self.cross_entropy_loss(y, y_pred)
            self.backward(X, y)
            if self.verbose and epoch % 100 == 0:
                print(f"Epoch {epoch} - Loss: {loss:.4f}")

    def predict(self, X):
        y_pred = self.forward(X)
        return (y_pred > 0.5).astype(int)
        

In [5]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values.reshape(-1, 1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

bcnn = BCNN(layers = [10,7,4],epochs = 2000, verbose = True, learning_rate=0.1)
bcnn.fit(X_train_scaled, y_train)

y_pred = bcnn.predict(X_test_scaled)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Epoch 0 - Loss: 0.7002
Epoch 100 - Loss: 0.6345
Epoch 200 - Loss: 0.5770
Epoch 300 - Loss: 0.5107
Epoch 400 - Loss: 0.4557
Epoch 500 - Loss: 0.4175
Epoch 600 - Loss: 0.3908
Epoch 700 - Loss: 0.3708
Epoch 800 - Loss: 0.3553
Epoch 900 - Loss: 0.3434
Epoch 1000 - Loss: 0.3337
Epoch 1100 - Loss: 0.3255
Epoch 1200 - Loss: 0.3178
Epoch 1300 - Loss: 0.3103
Epoch 1400 - Loss: 0.3027
Epoch 1500 - Loss: 0.2947
Epoch 1600 - Loss: 0.2864
Epoch 1700 - Loss: 0.2782
Epoch 1800 - Loss: 0.2702
Epoch 1900 - Loss: 0.2626
[[29  4]
 [ 2 26]]
              precision    recall  f1-score   support

           0       0.94      0.88      0.91        33
           1       0.87      0.93      0.90        28

    accuracy                           0.90        61
   macro avg       0.90      0.90      0.90        61
weighted avg       0.90      0.90      0.90        61

